# Getting Started with PyFixest

In a first step, we load the module and some example data:

In [1]:
%load_ext autoreload
%autoreload 2

from pyfixest.estimation import feols, fepois
from pyfixest.summarize import summary, etable
from pyfixest.visualize import coefplot, iplot
from pyfixest.utils import get_data

In [2]:
data = get_data()
data.head()

,Y,Y2,X1,X2,f1,f2,f3,group_id,Z1,Z2
0,NaN,-9.902164,0.0,0.457858,9.0,15.0,0.0,1.0,-0.302072,-0.786966
1,2.486016,-3.571090,NaN,-4.998406,8.0,6.0,9.0,11.0,NaN,-3.482305
2,1.817729,-3.353401,2.0,1.558480,NaN,11.0,0.0,18.0,1.698868,1.988056
3,5.179868,14.696121,2.0,1.560402,15.0,1.0,4.0,15.0,5.068548,2.063131
4,1.193511,-6.568647,2.0,-3.472232,20.0,19.0,9.0,5.0,0.315810,-2.827332


## OLS Estimation

We can estimate a fixed effects regression via the `feols()` function. `feols()` has three arguments: a two-sided model formula, the data, and optionally, the type of inference.

In [3]:
fit = feols(fml="Y~X1 | f1", data=data, vcov="HC1")
type(fit)

pyfixest.feols.Feols

The first part of the formula contains the dependent variable and "regular" covariates, while the second part contains fixed effects.

`feols()` returns an instance of the `Fixest` class.

To inspect the results, we can use a summary function or method:

In [4]:
fit.summary()

###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.424 |        0.058 |     7.330 |      0.000 |   0.310 |    0.538 |
---
RMSE: 1.421   R2: 0.371   R2 Within: 0.052


Alternatively, the `.summarize` module contains a `summary` function, which can be applied on instances of regression model objects 
or lists of regression model objects. 

In [5]:
summary(fit)

###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.424 |        0.058 |     7.330 |      0.000 |   0.310 |    0.538 |
---
RMSE: 1.421   R2: 0.371   R2 Within: 0.052


You can access individual elements of the summary via dedicated methods: `.tidy()` returns a "tidy" `pd.DataFrame`, 
`.coef()` returns estimated parameters, and `se()` estimated standard errors. Other methods include `pvalue()`, `confint()`
and `tstat()`.

In [6]:
fit.coef()

Coefficient
X1    0.424004
Name: Estimate, dtype: float64

In [7]:
fit.se()

Coefficient
X1    0.057843
Name: Std. Error, dtype: float64

## Standard Errors and Inference

Supported covariance types are "iid", "HC1-3", CRV1 and CRV3 (up to two-way clustering). Inference can be adjusted "on-the-fly" via the
`.vcov()` method:

In [8]:
fit.vcov({"CRV1": "group_id + f1"}).summary()
fit.vcov({"CRV3": "group_id"}).summary()

###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  CRV1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.424 |        0.076 |     5.598 |      0.000 |   0.265 |    0.583 |
---
RMSE: 1.421   R2: 0.371   R2 Within: 0.052
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  CRV3
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.424 |        0.076 |     5.590 |      0.000 |   0.265 |    0.583 |
---
RMSE: 1.421   R2: 0.371   R2 Within: 0.052


It is also possible to run a wild (cluster) bootstrap after estimation (via the [wildboottest module](https://github.com/s3alfisc/wildboottest)):

In [9]:
fit2 = feols(fml="Y~ X1", data=data, vcov={"CRV1": "group_id"})
fit2.wildboottest(param="X1", B=999)

param                            X1
t value                    5.187938
Pr(>|t|)                   0.002002
bootstrap_type                   11
inference         CRV(['group_id'])
impose_null                    True
dtype: object

Note that the wild bootstrap currently does not support fixed effects in the regression model. Supporting fixed effects is work in progress.

## IV Estimation 

It is also possible to estimate instrumental variable models with *one* endogenous variable and (potentially multiple) instruments:

In [10]:
iv_fit = feols(fml="Y2~ 1 | f1 + f2 | X1 ~ Z1 + Z2", data=data)
iv_fit.summary()

###

Estimation:  IV
Dep. var.: Y2, Fixed effects: f1+f2
Inference:  CRV1
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.195 |        0.406 |     0.479 |      0.636 |  -0.636 |    1.026 |
---


If the model does not contain any fixed effects, just drop the second part of the formula above:

In [11]:
feols(fml="Y~ 1 | X1 ~ Z1 + Z2", data=data).summary()

###

Estimation:  IV
Dep. var.: Y
Inference:  iid
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| Intercept     |      2.091 |        0.125 |    16.763 |      0.000 |   1.846 |    2.336 |
| X1            |      0.469 |        0.107 |     4.383 |      0.000 |   0.259 |    0.679 |
---


IV estimation with multiple endogenous variables and multiple estimation syntax is currently not supported. The syntax is "depvar ~ exog.vars | fixef effects | endog.vars ~ instruments".

## Poisson Regression 

With version `0.8.4`, it is possible to estimate Poisson Regressions (not yet on PyPi): 

In [12]:
from pyfixest.utils import get_data

pois_data = get_data(model="Fepois")
pois_fit = fepois(fml="Y~X1 | f1+f2", data=pois_data, vcov={"CRV1": "group_id"})
pois_fit.summary()

###

Estimation:  Poisson
Dep. var.: Y, Fixed effects: f1+f2
Inference:  CRV1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |     -0.012 |        0.035 |    -0.333 |      0.739 |  -0.079 |    0.056 |
---
Deviance: 1067.961


## Multiple Estimation 

`PyFixest` supports a range of multiple estimation functionality: `sw`, `sw0`, `csw`, `csw0`, and multiple dependent variables. If multiple regression syntax is used, 
`feols()` and `fepois` returns an instance of a `FixestMulti` object, which essentially consists of a dicionary of `Fepois` or `Feols` instances.

In [13]:
multi_fit = feols(fml="Y~X1 | csw0(f1, f2)", data=data, vcov="HC1")
multi_fit

In [14]:
multi_fit.summary()

###

Estimation:  OLS
Dep. var.: Y
Inference:  HC1
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| Intercept     |      2.181 |        0.090 |    24.141 |      0.000 |   2.004 |    2.359 |
| X1            |      0.382 |        0.070 |     5.485 |      0.000 |   0.245 |    0.519 |
---
RMSE: 1.765   R2: 0.03
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.424 |        0.058 |     7.330 |      0.000 |   0.310 |    0.538 |
---
RMSE: 1.421   R2: 0.371   R2 Within: 0.052
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1+f2
Inference:  HC1
Observations:  997

| Coefficient   |   Estimate

Alternatively, you can look at the estimation results via the `etable()` method:

In [15]:
multi_fit.etable()

fml              Y~X1        Y~X1|f1 Y~X1|f1+f2
Coefficient Intercept     X1      X1         X1
Estimate        2.181  0.382   0.424      0.430
Std. Error      0.090  0.070   0.058      0.049
t value        24.141  5.485   7.330      8.701
Pr(>|t|)        0.000  0.000   0.000      0.000
2.5 %           2.004  0.245   0.310      0.333
97.5 %          2.359  0.519   0.538      0.527

If you are only insterested in some parameters, e.g. "X1", you can use the following syntax:

In [16]:
multi_fit.etable().xs("X1", level=1, axis=1)

fml,Y~X1,Y~X1|f1,Y~X1|f1+f2
Estimate,0.382,0.424,0.430
Std. Error,0.070,0.058,0.049
t value,5.485,7.330,8.701
Pr(>|t|),0.000,0.000,0.000
2.5 %,0.245,0.310,0.333
97.5 %,0.519,0.538,0.527


You can access an individual model by its name - i.e. a formula - via the `all_fitted_models` attribure.

In [17]:
multi_fit.all_fitted_models["Y~X1"].tidy()

,Estimate,Std. Error,t value,Pr(>|t|),2.5 %,97.5 %
Coefficient,,,,,,
Intercept,2.181441,0.090362,24.141261,0.000000e+00,2.004120,2.358762
X1,0.381912,0.069623,5.485410,5.230771e-08,0.245287,0.518537


or equivalently via the `fetch_model` method:

In [18]:
multi_fit.fetch_model(0).tidy()

Model:  Y~X1


,Estimate,Std. Error,t value,Pr(>|t|),2.5 %,97.5 %
Coefficient,,,,,,
Intercept,2.181441,0.090362,24.141261,0.000000e+00,2.004120,2.358762
X1,0.381912,0.069623,5.485410,5.230771e-08,0.245287,0.518537


Here, `0` simply fetches the first model stored in the `all_fitted_models` dictionary, `1` the second etc.

Objects of type `Fixest` come with a range of additional methods: `tidy()`, `coef()`, `vcov()` etc, which 
essentially loop over the equivalent methods of all fitted models. E.g. `Fixest.vcov()` updates inference for all 
models stored in `Fixest`.

In [19]:
multi_fit.vcov("iid").summary()

###

Estimation:  OLS
Dep. var.: Y
Inference:  iid
Observations:  998

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| Intercept     |      2.181 |        0.091 |    23.864 |      0.000 |   2.002 |    2.361 |
| X1            |      0.382 |        0.069 |     5.504 |      0.000 |   0.246 |    0.518 |
---
RMSE: 1.765   R2: 0.03
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1
Inference:  iid
Observations:  997

| Coefficient   |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5 % |   97.5 % |
|:--------------|-----------:|-------------:|----------:|-----------:|--------:|---------:|
| X1            |      0.424 |        0.057 |     7.393 |      0.000 |   0.311 |    0.537 |
---
RMSE: 1.421   R2: 0.371   R2 Within: 0.052
###

Estimation:  OLS
Dep. var.: Y, Fixed effects: f1+f2
Inference:  iid
Observations:  997

| Coefficient   |   Estimate

If you have estimated multiple models without multiple estimation syntax and still want to compare them, you can use the `etable()` function: 

In [20]:
from pyfixest.summarize import etable

etable([fit, fit2])

                          est1              est2
------------  ----------------  ----------------
depvar                       Y                 Y
------------------------------------------------
X1            0.424*** (0.076)  0.382*** (0.074)
Intercept                       2.181*** (0.086)
------------------------------------------------
f1                           x                 -
------------------------------------------------
Observations               997               998
S.E. type         by: group_id      by: group_id
------------------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001


## Visualization 

`PyFixest` provides two functions to visualize the results of a regression: `coefplot` and `iplot`.

In [21]:
from lets_plot import *

LetsPlot.setup_html()

multi_fit.coefplot().show()

## Difference-in-Differences / Event Study Designs

`PyFixest` supports eventy study designs via two-way fixed effects and Gardner's 2-stage estimator. 

In [22]:
import pandas as pd
import numpy as np
from pyfixest.did.did import did2s

file_path = "../pyfixest/experimental/data/df_het.csv"
df_het = pd.read_csv(file_path)
df_het.head()


      You have loaded the 'pyfixest.did' module. Code in this module is not yet stable and
      may change in the future. It may also not be sufficiently unit tested (if at all) or documented.
      


,unit,state,group,unit_fe,g,year,year_fe,treat,rel_year,rel_year_binned,error,te,te_dynamic,dep_var
0,1,33,Group 2,7.043016,2010,1990,0.066159,False,-20.0,-6,-0.086466,0,0.0,7.022709
1,1,33,Group 2,7.043016,2010,1991,-0.030980,False,-19.0,-6,0.766593,0,0.0,7.778628
2,1,33,Group 2,7.043016,2010,1992,-0.119607,False,-18.0,-6,1.512968,0,0.0,8.436377
3,1,33,Group 2,7.043016,2010,1993,0.126321,False,-17.0,-6,0.021870,0,0.0,7.191207
4,1,33,Group 2,7.043016,2010,1994,-0.106921,False,-16.0,-6,-0.017603,0,0.0,6.918492


In [23]:
fit_did2s = did2s(
    df_het,
    yname="dep_var",
    first_stage="~ 0 | state + year",
    second_stage="~i(rel_year)",
    treatment="treat",
    cluster="state",
    i_ref1=[-1.0, np.inf],
)

fit_twfe = feols(
    "dep_var ~ i(rel_year) | state + year",
    df_het,
    i_ref1=[-1.0, np.inf],
    vcov={"CRV1": "state"},
)

iplot(
    [fit_did2s, fit_twfe], coord_flip=False, figsize=(900, 400), title="TWFE vs DID2S"
)

c:\Users\alexa\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:394: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
c:\Users\alexa\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


The `event_study()` function provides a common API for several event study estimators.

In [24]:
from pyfixest.did.did import event_study
from pyfixest.summarize import etable

fit_twfe = event_study(
    data=df_het,
    yname="dep_var",
    idname="state",
    tname="year",
    gname="g",
    estimator="twfe",
)

fit_did2s = event_study(
    data=df_het,
    yname="dep_var",
    idname="state",
    tname="year",
    gname="g",
    estimator="did2s",
)

etable([fit_twfe, fit_did2s])

                          est1              est2
------------  ----------------  ----------------
depvar                 dep_var       dep_var_hat
------------------------------------------------
ATT           2.135*** (0.044)  2.152*** (0.048)
------------------------------------------------
state                        x                 -
year                         x                 -
------------------------------------------------
Observations             46500             46500
S.E. type            by: state              CRV1
------------------------------------------------
Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001
